1. Data Preparation
2. Machine Learning Modelling
3. Model Evaluation
4. Export Trained Model

In [29]:
import pandas as pd

In [30]:
data = pd.read_csv("../input/age-of-marriage/age_of_marriage_data.csv")
print(data.shape)
data.head(5)

(2567, 10)


,id,gender,height,religion,caste,mother_tongue,profession,location,country,age_of_marriage
0,1,female,"5'4""",NaN,others,Telugu,NaN,London,United Kingdom,21.0
1,2,male,"5'7""",Jain,Shwetamber,Gujarati,Doctor / Healthcare Professional,Fairfax- VA,USA,32.0
2,3,male,"5'7""",Hindu,Brahmin,Hindi,Entrepreneurs / Business,Begusarai,India,32.0
3,4,female,"5'0""",Hindu,Thakur,Hindi,Architect,Mumbai,India,30.0
4,5,male,"5'5""",Christian,Born Again,Malayalam,Sales Professional / Marketing,Sulthan Bathery,India,30.0


In [31]:
print(data.isnull().sum())

id                   0
gender              29
height             118
religion           635
caste              142
mother_tongue      164
profession         330
location           155
country             16
age_of_marriage     19
dtype: int64


In [32]:
missing_percentage = 1 - (data.dropna().shape[0]/data.shape[0])
print(missing_percentage)

0.2473704713673549


In [33]:
data.dropna(inplace=True)

In [34]:
data

,id,gender,height,religion,caste,mother_tongue,profession,location,country,age_of_marriage
1,2,male,"5'7""",Jain,Shwetamber,Gujarati,Doctor / Healthcare Professional,Fairfax- VA,USA,32.0
2,3,male,"5'7""",Hindu,Brahmin,Hindi,Entrepreneurs / Business,Begusarai,India,32.0
3,4,female,"5'0""",Hindu,Thakur,Hindi,Architect,Mumbai,India,30.0
4,5,male,"5'5""",Christian,Born Again,Malayalam,Sales Professional / Marketing,Sulthan Bathery,India,30.0
5,6,male,"5'5""",Hindu,Valmiki,Hindi,Sportsman,Delhi,India,29.0
...,...,...,...,...,...,...,...,...,...,...
2561,2562,male,"5'11""",Hindu,Kshatriya,Odia,VP / AVP / GM / DGM,Mumbai,India,30.0
2562,2563,female,"5'3""",Hindu,Thakur,Hindi,Banking Professional,Agra,India,27.0
2563,2564,male,"5'11""",Hindu,Thakur,Hindi,Software Professional,Ottawa,Canada,31.0
2564,2565,female,"5'3""",Hindu,Baishya,Bengali,Software Consultant,Bengaluru / Bangalore,India,28.0


In [35]:
data.shape

(1932, 10)

In [36]:
X = data.loc[:,['gender','height','religion','caste','mother_tongue','country']]
y = data.iloc[:,-1]


In [37]:
X.head()

,gender,height,religion,caste,mother_tongue,country
1,male,"5'7""",Jain,Shwetamber,Gujarati,USA
2,male,"5'7""",Hindu,Brahmin,Hindi,India
3,female,"5'0""",Hindu,Thakur,Hindi,India
4,male,"5'5""",Christian,Born Again,Malayalam,India
5,male,"5'5""",Hindu,Valmiki,Hindi,India


In [39]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
X.loc[:,['gender','religion','caste','mother_tongue','country']]= \
                X.loc[:,['gender','religion','caste','mother_tongue','country']].apply(enc.fit_transform)

In [40]:
X.head()

,gender,height,religion,caste,mother_tongue,country
1,1,"5'7""",2,34,6,19
2,1,"5'7""",1,14,8,5
3,0,"5'0""",1,36,8,5
4,1,"5'5""",0,13,13,5
5,1,"5'5""",1,38,8,5


In [41]:
def h_cms(h):
    return int(h.split('\'')[0])*30.48+\
    int(h.split('\'')[1].replace('"',''))*2.54

In [42]:
X['height_cms'] = X.height.apply(h_cms)

In [43]:
X.head()

,gender,height,religion,caste,mother_tongue,country,height_cms
1,1,"5'7""",2,34,6,19,170.18
2,1,"5'7""",1,14,8,5,170.18
3,0,"5'0""",1,36,8,5,152.40
4,1,"5'5""",0,13,13,5,165.10
5,1,"5'5""",1,38,8,5,165.10


In [45]:
X.drop('height',inplace=True,axis=1)

In [46]:
X

,gender,religion,caste,mother_tongue,country,height_cms
1,1,2,34,6,19,170.18
2,1,1,14,8,5,170.18
3,0,1,36,8,5,152.40
4,1,0,13,13,5,165.10
5,1,1,38,8,5,165.10
...,...,...,...,...,...,...
2561,1,1,21,18,5,180.34
2562,0,1,36,8,5,160.02
2563,1,1,36,8,3,180.34
2564,0,1,7,2,5,160.02


In [47]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [48]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=80,max_depth=11)
model.fit(X_train,y_train)
y_predict = model.predict(X_test)

In [49]:
from sklearn.metrics import mean_absolute_error, r2_score
print("MAE : ", mean_absolute_error(y_test,y_predict))
r2_score(y_test,y_predict)

MAE :  1.0329329901973245


0.7014353208990445

In [53]:
import pickle
pickle.dump(model,open('marriage_age_predict_model.ml','wb'))